In [ ]:
import numpy as np
import random
import copy
from itertools import permutations

In [ ]:
class InitializationHeuristics:
  def __init__(self, cities):
    self.cities = cities
    self.distance_matrix = self.distance_matrix()
    self.final_tour = None
    self.tota_cost = None
    self.run()

  def run(self):
    input_list = list(range(len(self.cities)))

    # Bước 1: Chọn các thành phố đặc biệt
    extreme_cities = set()
    extreme_cities.add(max(input_list, key=lambda i: self.cities[i][0]))
    extreme_cities.add(min(input_list, key=lambda i: self.cities[i][0]))
    extreme_cities.add(max(input_list, key=lambda i: self.cities[i][1]))
    extreme_cities.add(min(input_list, key=lambda i: self.cities[i][1]))

    # Danh sách vị trí các thành phố đặc biệt
    output_list = list(extreme_cities)
    for city_index in output_list:
      input_list.remove(city_index)

    # Bước 2: Tìm thứ tự tốt nhất cho các thành phố đã chọn
    output_list = self.find_best_order(output_list)

    # Bước 3: Xáo trộn danh sách Input
    random.shuffle(input_list)

    # Bước 4 và 5: Chèn các thành phố còn lại vào vị trí tốt nhất
    while input_list:
        city = input_list.pop(0)
        best_position = None
        min_increase = float('inf')

        for i in range(len(output_list)):
            new_tour = output_list[:i] + [city] + output_list[i:]
            increase = self.total_tour_cost(new_tour) - self.total_tour_cost(output_list)
            if increase < min_increase:
                min_increase = increase
                best_position = i

        output_list.insert(best_position, city)

    self.final_tour = output_list
    self.tota_cost = self.total_tour_cost(self.final_tour)

  def find_best_order(self, four_cities):
    # Hàm tìm thứ tự tối ưu cho 4 thành phố đầu tiên
    best_order = None
    min_cost = float('inf')

    for tour in permutations(four_cities):  # Hoán vị trí các thành phố
      cost = self.total_tour_cost(tour)     # Mỗi hoán vị là một tour
      if cost < min_cost:
        min_cost = cost
        best_order = tour

    return list(best_order)

  def euclidean_distance(self, city1, city2):
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

  def total_tour_cost(self, tour):
    # Tổng quảng đường du lịch đi từ thành phố này đến thành phố khác trong tour,
    cost = 0
    for i in range(len(tour)):
        cost += self.euclidean_distance(
                  self.cities[tour[i]],
                  self.cities[tour[(i + 1) % len(tour)]]
                )
    return cost

  def distance_matrix(self):
    n = len(self.cities)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distance_matrix[i][j] = self.euclidean_distance(self.cities[i], self.cities[j])
    return distance_matrix

In [ ]:
class RemoveSharp:
  def __init__(self, tour, distance_matrix, num_near_cities):
    self.distance_matrix = distance_matrix
    self.num_near_cities = num_near_cities
    self.tour = tour
    self.best_tour = None
    self.best_cost = None
    self.run()

  def run(self):
    n_tour = len(self.tour)
    self.best_tour = self.tour.copy()
    self.best_cost = self.total_tour_cost(self.tour, self.distance_matrix)

    for i in range(n_tour):
      city_index = self.tour[i]
      prev_city_index = self.tour[i - 1] if i > 0 else self.tour[-1]
      next_city_index = self.tour[(i + 1) % n_tour]

      # Có cần thiết phải đi qua city_index khi
      # đi từ prev_city_index đến next_city_index không (nếu có thì < 0)?
      decrease_distance = (self.distance_matrix[prev_city_index, city_index] +
                           self.distance_matrix[city_index, next_city_index] -
                           self.distance_matrix[prev_city_index, next_city_index])

      new_tour = self.tour[:i] + self.tour[i+1:] # Loại bỏ city thứ i ra khỏi tour hiện tại
      near_cities_index = self.near_cities(city_index, new_tour)

      for near_city_index in near_cities_index:
        prev_near_city_index = new_tour[near_city_index - 1] if near_city_index > 0 else new_tour[-1]
        next_near_city_index = new_tour[(near_city_index + 1) % len(new_tour)]

        # Có cần thiết phải đi qua city_index khi
        # đi từ prev_near_city_index đến near_city_index không (nếu có thì < 0)?
        increase_before = (self.distance_matrix[prev_near_city_index, city_index] +
                           self.distance_matrix[city_index, near_city_index] -
                           self.distance_matrix[prev_near_city_index, near_city_index])

        # Có cần thiết phải đi qua city_index khi
        # đi từ near_city_index đến next_near_city_index không (nếu có thì < 0) ?
        increase_after = (self.distance_matrix[near_city_index, city_index] +
                          self.distance_matrix[city_index, next_near_city_index] -
                          self.distance_matrix[near_city_index, next_near_city_index])

        # Trường hợp 1
        # prev_near_city_index -> near_city_index ngắn hơn
        # prev_city_index -> next_city_index
        # Chọn tuyến đi qua nhà hàng xóm (sân trước) :))
        if increase_before < decrease_distance:
          modified_tour = new_tour[:near_city_index] + [city_index] + new_tour[near_city_index:]
          new_cost = self.total_tour_cost(modified_tour, self.distance_matrix)
          if new_cost < self.best_cost:
            self.best_cost = new_cost
            self.best_tour = modified_tour.copy()

        # Trường hợp 2
        # near_city_index -> next_near_city_index ngắn hơn
        # prev_city_index -> next_city_index
        # Chọn tuyến đi qua nhà hàng xóm (sân sau) :))
        if increase_after < decrease_distance:
          modified_tour = new_tour[:near_city_index + 1] + [city_index] + new_tour[near_city_index + 1:]
          new_cost = self.total_tour_cost(modified_tour, self.distance_matrix)
          if new_cost < self.best_cost:
            self.best_cost = new_cost
            self.best_tour = modified_tour.copy()

  def near_cities(self, city_index, tour):
    # Tính toán khoảng cách từ city -> mọi city thuộc tour
    distances = [(i, self.distance_matrix[city_index, tour[i]])
                 for i in range(len(tour))]
    distances.sort(key=lambda x: x[1])
    # Index của num_near_cities city thuộc tour gần với city nhất
    return [distance[0] for distance in distances[:self.num_near_cities]]

  def total_tour_cost(self, tour, dist_matrix):
    cost = 0
    for i in range(len(tour)-1):
      cost += dist_matrix[tour[i], tour[i+1]]
    cost += dist_matrix[tour[-1], tour[0]]
    return cost


In [ ]:
class Crossover:
  def __init__(self):
    self.offspring = None
    self.edge_map = None

  def build_edge_map(self, parent1, parent2):
    unique_parent = set(parent1) | set(parent2)
    edge_map = {city: set() for city in unique_parent}

    for parent in (parent1, parent2):
      size = len(parent)
      for i in range(size):
        city = parent[i]
        left_neighbor = parent[i - 1] if i > 0 else parent[size - 1]
        right_neighbor = parent[(i + 1) % size]
        edge_map[city].update([left_neighbor, right_neighbor])

    return edge_map

  def select_next_city(self, current_city, edge_map):
    if edge_map[current_city]:
      return min(edge_map[current_city], key=lambda city: len(edge_map[city]))
    return None

  def __call__(self, parent1, parent2):
    current_city = random.choice(parent1)
    self.offspring = [current_city]
    self.edge_map = self.build_edge_map(parent1, parent2)
    edge_map = copy.deepcopy(self.edge_map)

    while len(self.offspring) < len(parent1):
      for city in edge_map:
        edge_map[city].discard(current_city)

      next_city = self.select_next_city(current_city, edge_map)
      if next_city is None:
        remaining_cities = list(set(parent1) - set(self.offspring))
        if remaining_cities:
          next_city = random.choice(remaining_cities)
        else:
          break

      self.offspring.append(next_city)
      current_city = next_city

    return self

In [ ]:
class LocalOpt:
  def __init__(self, tour, distance_matrix, num_consecutive_cities):
    self.tour = tour
    self.distance_matrix = distance_matrix
    self.num_consecutive_cities = num_consecutive_cities
    self.best_tour = None
    self.best_cost = None
    self.run()

  def run(self):
    n = len(self.tour)
    self.best_tour = self.tour.copy()
    self.best_cost = self.total_tour_cost(self.best_tour, self.distance_matrix)

    for i in range(n):
      sub_tour = self.tour[i:i+self.num_consecutive_cities]
      if len(sub_tour) < self.num_consecutive_cities:
        sub_tour += self.tour[:self.num_consecutive_cities-len(sub_tour)]

      for perm in permutations(sub_tour):
        new_tour = self.tour.copy()
        new_tour[i:i+self.num_consecutive_cities] = list(perm)
        if i + self.num_consecutive_cities > n:
          new_tour[:self.num_consecutive_cities-len(sub_tour)] = list(perm[-self.num_consecutive_cities+n:])
        new_cost = self.total_tour_cost(new_tour, self.distance_matrix)

        if new_cost < self.best_cost:
          self.best_tour = new_tour.copy()
          self.best_cost = new_cost

  def total_tour_cost(self, tour, dist_matrix):
    cost = 0
    for i in range(len(tour)-1):
      cost += dist_matrix[tour[i], tour[i+1]]
    cost += dist_matrix[tour[-1], tour[0]]
    return cost

In [ ]:
class HybridGenetic:
  def __init__(self, num_near_cities, num_consecutive_cities,
               crossover = Crossover, remove_sharp = RemoveSharp, local_opt = LocalOpt,
               initialization_heuristics = InitializationHeuristics):

    self.num_consecutive_cities = num_consecutive_cities
    self.num_near_cities = num_near_cities
    self.crossover = crossover()
    self.remove_sharp = remove_sharp
    self.local_opt = local_opt
    self.initialization_heuristics = initialization_heuristics

  def total_tour_cost(self, tour, dist_matrix):
    cost = 0
    for i in range(len(tour)-1):
      cost += dist_matrix[tour[i], tour[i+1]]
    cost += dist_matrix[tour[-1], tour[0]]
    return cost

  def __call__(self, n_cities, pop_size, num_iterations):

    # Step 1: Khởi tạo quần thể
    population = []

    cities = np.random.rand(n_cities, 2) * 100
    IH = self.initialization_heuristics(cities)
    tour = IH.final_tour
    population.append(tour)

    # Khởi tạo ngẫu nhiên
    for _ in range(pop_size - 1):
      tour = random.sample(range(len(population[0])), len(population[0]))
      population.append(tour)

    # Step 2: Áp dụng RemoveSharp và LocalOpt
    for i in range(pop_size):
      RS = self.remove_sharp(population[i], IH.distance_matrix, self.num_near_cities)
      LO = self.local_opt(RS.best_tour, IH.distance_matrix, self.num_consecutive_cities)
      population[i] = LO.best_tour

    # Step 3 & 4: Tiến hóa qua nhiều thế hệ
    for _ in range(num_iterations):
      parent1, parent2 = random.sample(population, 2)
      offspring = self.crossover(parent1, parent2).offspring
      RS = self.remove_sharp(offspring, IH.distance_matrix, self.num_near_cities)
      LO = self.local_opt(RS.best_tour, IH.distance_matrix, self.num_consecutive_cities)
      offspring = LO.best_tour

      # Thay thế cha mẹ yếu hơn nếu con tốt hơn
      parent_cost = max(self.total_tour_cost(parent1, IH.distance_matrix),
                        self.total_tour_cost(parent2, IH.distance_matrix))
      if self.total_tour_cost(offspring, IH.distance_matrix) < parent_cost:
        if self.total_tour_cost(parent1, IH.distance_matrix) > self.total_tour_cost(parent2, IH.distance_matrix):
          population.remove(parent1)
        else:
          population.remove(parent2)

        population.append(offspring)

      # Step 4: Xáo trộn một cá thể bất kỳ trong quần thể
      random.shuffle(population[random.randint(0, pop_size - 1)])

    # Trả về tour tốt nhất tìm được
    return min(population, key=lambda tour: self.total_tour_cost(tour, IH.distance_matrix))

In [ ]:
HG = HybridGenetic(num_near_cities=10, num_consecutive_cities=3)

In [ ]:
HG(10, 100, 100)

[0, 1, 5, 2, 8, 9, 4, 6, 3, 7]